# Exercise Session 2

# Exercises

### 1- Modeling with RDF
a. It is possible to reach Amsterdam either by train, by bus, or by plane.<br>
b. Amsterdam can only be reached by train, by bus or by plane.

### Answer

a.<br>

    @prefix : <http://ex.org/> .
    :Amsterdam :canBeReachedVia _:b .  
    _:b rdf:type rdf:Alt .  
    _:b rdf:_1 :Train .  
    _:b rdf:_2 :Bus .  
    _:b rdf:_3 :Plane .  

b.<br>

    :Amsterdam :canBeReachedVia _:b1 . 
    _:b1 rdf:first :Train .  
    _:b1 rdf:rest _:b2 . 
    _:b2 rdf:first :Bus .  
    _:b2 rdf:rest _:b3 .  
    _:b3 rdf:first :Plane .  
    _:b3 rdf:rest rdf:nil .  

  Or

    :Amsterdam :canBeReachedVia ( :Train :Bus :Plane ) . 


### 2- Create an RDF model that represents the following statements: <br> 
a). <http://ex.org/Resource/Document1> is created by John.  <br>
b). <http://ex.org/Resource/Document2> and <http://ex.org/Resource/Document3> are created by an unknown author. <br>
c). <http://ex.org/Document3> says that <http://ex.org/Document1> is published by W3C.

You can make of use of the predicates http://purl.org/dc/elements/1.1/creator  and http://purl.org/dc/elements/1.1/publisher to create the model.
        

### Answer

    @prefix : <http://ex.org/Resource/> . 
    @prefix dc: <http://purl.org/dc/elements/1.1/> . 
    :Document1 dc:creator "John" . 
    :Document2 dc:creator _:b1 . 
    :Document3 dc:creator _:b1 . 
    :Document3 dc:claims _:b2 . 
    _:b2 rdf:type rdf:Statement .  
    _:b2 rdf:subject :Document1 . 
    _:b2 rdf:predicate dc:publisher . 
    _:b2 rdf:object "W3C" . 

### 3- Using FOAF

Create your own FOAF profile using the Turtle syntax.

FOAF = friend of a friend
-  FOAF is a vocabulary to describe people and their social networks. <br>
-  Prefix foaf: <http://xmlns.com/foaf/0.1/>

a) Create your own FOAF profile using the Turtle syntax. <br>
b) Convert the Turtle syntax into RDF/XML.

You can use the FOAF generator http://www.ldodds.com/foaf/foaf-a-matic.html to verify
your results.

### Answer

    @prefix foaf: <http://xmlns.com/foaf/0.1/> .

    <http://john.me/#me>  
      a foaf:Person ; 
      foaf:name "John Smith" ; 
      foaf:title "Dr" ; 
      foaf:givenname "John" ; 
      foaf:family_name "Smith" ; 
      foaf:nick "Jo" ; 
      foaf:homepage <http://john.me> ;
      foaf:depiction <http://john.me/do.jpg> ; 
      foaf:mbox <mailto:john@smith.com> ;  
      foaf:phone "000111222333" .  

In [4]:
from rdflib import Graph
g = Graph()
g.parse("myfoaf.ttl", format='turtle')
print(g.serialize(format="xml"))

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:foaf="http://xmlns.com/foaf/0.1/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://john.me/#me">
    <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
    <foaf:name>John Smith</foaf:name>
    <foaf:title>Dr</foaf:title>
    <foaf:givenname>John</foaf:givenname>
    <foaf:family_name>Smith</foaf:family_name>
    <foaf:nick>Jo</foaf:nick>
    <foaf:homepage rdf:resource="http://john.me"/>
    <foaf:depiction rdf:resource="http://john.me/do.jpg"/>
    <foaf:mbox rdf:resource="mailto:john@smith.com"/>
    <foaf:phone>000111222333</foaf:phone>
  </rdf:Description>
</rdf:RDF>



### 4- Reification

Create an RDF graph that represents the following statement:

John claims that the creator of the resource http://www.amazon.com is Jeff Bezos. 

### Answer

     @prefix : <http://ex.org/> .
     :John :claims _:b.  
    _:b rdf:type rdf:Statement .  
    _:b rdf:subject :JeffBezos .  
    _:b rdf:predicate :creator .  
    _:b rdf:object <http://www.amazon.com> .

### 5- Searching RDF graphs through programming  

Given the following RDF graph, write a Python program to extend it with additional triples. Retrieve these triples from DBpedia, focusing on Utrecht's population, latitude, and longitude, along with the country to which it belongs. This extension allows us to answers the query "what is the population of the city in which Utrecht University, offering the course in Knowledge Engineering, is located?" 

    @prefix ex: <http://ex.org/>
    @prefix db: <https://dbpedia.org/resource/>
    ex:KDE ex:offeredAt ex:UtrechtUniversity . 
    ex:UtrechtUniversity ex:locatedIn db:Utrecht .

### Answer

In [17]:
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import Namespace, XSD

ex = Namespace("http://ex.org/")
db = Namespace("https://dbpedia.org/resource/")

g = Graph()
g.add((ex["KDE"], ex["offeredAt"], ex["UtrechtUniversity"]))
g.add((ex["UtrechtUniversity"], ex["locatedIn"], db["Utrecht"]))
# serialize and print the current graph 
print("Graph in Turtle syntax")
print("-----------------------")
print(g.serialize(format="turtle"))

dbo = Namespace("http://dbpedia.org/ontology/")
geo = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
g.bind("dbo", dbo)
g.bind("geo", geo)
g.bind("db", db)

g2 = Graph()
g2.parse(URIRef(db["Utrecht"]))
g2.serialize(format="turtle")
 
for s, p, o in g2: 
    if p == dbo.populationTotal:
        g.add((db.Utrecht, dbo.populationTotal, o))
    elif p == dbo.country:
        g.add((db.Utrecht,dbo.country, o))
    elif p == geo.lat:
        o = Literal(o, datatype=XSD.float)
        g.add((db.Utrecht, geo.lat, o))
    elif p == geo.long:
        o = Literal(o, datatype=XSD.float)
        g.add((db.Utrecht, geo.long, o))        
    else:
        continue

# count the number of triples in the extended graph 
print("Number of triples in the extended graph: ", len(g))
print("-----------------------")

# serialize and print the extended graph 
print("Extended graph")
print("-----------------------")

print(g.serialize(format="turtle"))


Graph in Turtle syntax
-----------------------
@prefix ns1: <http://ex.org/> .

ns1:KDE ns1:offeredAt ns1:UtrechtUniversity .

ns1:UtrechtUniversity ns1:locatedIn <https://dbpedia.org/resource/Utrecht> .


Number of triples in the extended graph:  6
-----------------------
Extended graph
-----------------------
@prefix db: <https://dbpedia.org/resource/> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix geo1: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ns1: <http://ex.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:KDE ns1:offeredAt ns1:UtrechtUniversity .

ns1:UtrechtUniversity ns1:locatedIn db:Utrecht .

db:Utrecht dbo:country <http://dbpedia.org/resource/Netherlands> ;
    dbo:populationTotal "361742"^^xsd:nonNegativeInteger ;
    geo1:lat "52.08333206176758"^^xsd:float ;
    geo1:long "5.116666793823242"^^xsd:float .




### 6- RDF Schema for Mythological and Legendary Figures

We will use RDFS to define some semantics for classes and properties relating to mythological and legendary figures and use these definitions to infer new data in the RDF graph. 

    @prefix ex:  <http://ex.org/>.
    @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
    ex:CuChulainn a ex:Genma ;
        ex:knownAs [ ex:name "Cú Chulainn"; ex:meaning "Culann's Hound"@en ] ;
        ex:knownAs [ ex:name "Sétanta" ] .
    ex:Ahriman a ex:Vile ;
        ex:knownAs [ ex:name "Ahriman" ] ;
        ex:knownAs [ ex:name "Angra Mainyu" ; ex:meaning "Destructive Spirit"@en ] .
    ex:Vishnu a ex:Deity ;
        ex:consort ex:Lakshmi ;
        ex:knownAs [ ex:name "Vishnu" ; ex:meaning "All pervasive"@en ] .
    ex:Lakshmi a ex:God ;
        ex:consort ex:Vishnu ;
        ex:commonlyKnownAs [ ex:name "Lakshmi" ; ex:meaning "Know and understand your goal"@en ] .
    ex:Vile ex:alignment ex:Lawful , ex:Evil .
    ex:Genma ex:alignment ex:LawfulNeutral .
    ex:LawfulGood a ex:Alignment .
    ex:LawfulEvil a ex:Alignment .
    
    
Note that RDF terms are case sensitive; the convention is to use lower (camel) case for properties, like ex:alignment and proper (camel) case for classes and entities, like ex:Alignment. The answer to each question should be kept and extended (some questions depend on previous answers).

1. Add one  RDFS (range) triple to infer:
    
        ex:Lawful a ex:Alignment .   
        ex:Evil a ex:Alignment .     
        ex:LawfulNeutral a ex:Alignment .
        
   Solution: 
   
     <span style="color:red">ex:alignment rdfs:range ex:Alignment .</span>
    
2. Add one RDFS (domain) triple to infer:
    
        ex:CuChulainn a ex:Entity .  
        ex:Ahriman a ex:Entity .     
        ex:Vishnu a ex:Entity .             
        
   Solution: 
   
     <span style="color:red">  ex:knownAs rdfs:domain ex:Entity .</span>
   
    
3. Add one RDFS (subproperty) triple to infer:
    
        ex:Vishnu ex:companion ex:Lakshmi .  
        ex:Lakshmi ex:companion ex:Vishnu .

   Solution: 
   
     <span style="color:red">  ex:consort rdfs:subPropertyOf ex:companion .</span>

### 7- RDFS inference through rdflib and owlrl

If you haven't done so already, install the packages rdflib and owlrl using:

    !pip install rdflib
    !pip install owlrl

* See the [rdflib documention](https://rdflib.readthedocs.io/en/stable/index.html) for full documentation on rdflib
* See the [owlrl documentation](https://owl-rl.readthedocs.io/en/latest/) for information on rdfs and owl-rl reasoning

For the questions from the above task (*Task 6*), provide code that allows to perform inference on the above ontology. 

#### import rdflib and owlrl
 * [owlrl](https://owl-rl.readthedocs.io/en/latest/) provides functions that allow to infer triples from RDFS  ontologies.

In [19]:
from rdflib import Graph, Namespace, Literal, URIRef
from owlrl.CombinedClosure import RDFS_OWLRL_Semantics
import owlrl

graph = """
@prefix ex:  <http://ex.org/>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
ex:CuChulainn a ex:Genma ;
    ex:knownAs [ ex:name "Cú Chulainn"; ex:meaning "Culann's Hound"@en ] ;
    ex:knownAs [ ex:name "Sétanta" ] .
ex:Ahriman a ex:Vile ;
    ex:knownAs [ ex:name "Ahriman" ] ;
    ex:knownAs [ ex:name "Angra Mainyu" ; ex:meaning "Destructive Spirit"@en ] .
ex:Vishnu a ex:Deity ;
    ex:consort ex:Lakshmi ;
    ex:knownAs [ ex:name "Vishnu" ; ex:meaning "All pervasive"@en ] .
ex:Lakshmi a ex:God ;
    ex:consort ex:Vishnu ;
    ex:commonlyKnownAs [ ex:name "Lakshmi" ; ex:meaning "Know and understand your goal"@en ] .
ex:Vile ex:alignment ex:Lawful , ex:Evil .
ex:Genma ex:alignment ex:LawfulNeutral .
ex:LawfulGood a ex:Alignment .
ex:LawfulEvil a ex:Alignment .

ex:alignment rdfs:range ex:Alignment .
"""

g = Graph().parse(data=graph, format="ttl")

In [20]:
# add to graph g (Ontology in Task 6) all triples inferred by the RDFS  
# Method 1
#rdfs_inference = RDFS_OWLRL_Semantics(g, False, False, False)
#rdfs_inference.closure()
#rdfs_inference.flush_stored_triples()

# or Method 2
owlrl.DeductiveClosure(owlrl.RDFS_Semantics).expand(g)

# or method 3 
#owlrl.RDFSClosure.RDFS_Semantics(g, False, False, False)

print(g.serialize(format="turtle"))

@prefix ex: <http://ex.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Ahriman a ex:Vile ;
    ex:knownAs [ ex:name "Ahriman" ],
        [ ex:meaning "Destructive Spirit"@en ;
            ex:name "Angra Mainyu" ] .

ex:CuChulainn a ex:Genma ;
    ex:knownAs [ ex:meaning "Culann's Hound"@en ;
            ex:name "Cú Chulainn" ],
        [ ex:name "Sétanta" ] .

ex:LawfulEvil a ex:Alignment .

ex:LawfulGood a ex:Alignment .

ex:alignment rdfs:range ex:Alignment .

ex:Genma ex:alignment ex:LawfulNeutral .

ex:Lakshmi a ex:God ;
    ex:commonlyKnownAs [ ex:meaning "Know and understand your goal"@en ;
            ex:name "Lakshmi" ] ;
    ex:consort ex:Vishnu .

ex:Vile ex:alignment ex:Evil,
        ex:Lawful .

ex:Vishnu a ex:Deity ;
    ex:consort ex:Lakshmi ;
    ex:knownAs [ ex:meaning "All pervasive"@en ;
            ex:name "Vishnu" ] .




### 8- modeling in RDFS

1. Expand the following RDFS ontology and data with additional axioms. 


        rdfs_ontology = """
        @prefix terms: <http://example.org/ontology/> .
        @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

        terms:Person a rdfs:Class; rdfs:subClassOf terms:Mammal .
        terms:spouse rdfs:domain terms:Person;
                rdfs:range terms:Person.
        """

        data = """
            @prefix ex: <http://example.org/> .
            @prefix terms: <http://example.org/ontology/> .
            ex:john terms:spouse ex:mary.
        """

2. Provide the triples that can be inferred from the extended graph using rdflib and owlrl. 

### 9- modeling in RDFS

-  Model the domain of films/cinematography using RDFS. 